In [3]:
#Importing Libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit
from hmmlearn import hmm
from tqdm import tqdm

In [4]:
#Data Importing
dataset = pd.read_csv("creditcard_2023.csv")
datasetdataset = dataset.fillna(method="ffill")
datasetdataset = dataset.rename(columns = { 'Sentence #' : 'sentence' })
dataset.head(5)

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [9]:
amt = list(set(dataset.Amount.values))
tags = list(set(dataset.Class.values))
len(tags) , len(amt)

(2, 506485)

In [18]:
y = dataset.Amount
x = dataset.drop( 'Amount', axis = 1 )
groupshufflesplit = GroupShuffleSplit( n_splits =  2 , test_size = .33 , random_state = 42 )
ix_train,ix_test = next(groupshufflesplit.split(x,y,groups= dataset['Class']))
dataset_train = dataset.loc[ix_train]
dataset_test = dataset.loc[ix_test]

dataset_train

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,284802,-1.605484,2.041786,-1.198693,-1.883651,-1.054493,-1.199907,-0.635763,1.849901,1.948540,...,-0.018546,0.060656,1.062051,-0.863202,2.407534,0.562133,1.024829,1.878089,15599.27,0
284803,284803,-0.080325,-0.463534,1.977771,-1.016889,0.643741,1.046862,0.441402,-0.084550,0.880137,...,-0.018251,0.857735,-0.011161,-1.658129,-1.036511,-0.958793,-0.158577,-0.229647,3614.52,0
284804,284804,1.653848,-0.550318,-0.382720,-0.914126,1.676939,2.007193,0.336910,0.014224,0.773796,...,-0.011261,0.509828,-0.061656,1.377016,0.354133,-0.189053,-0.232734,-0.164373,4366.80,0
284805,284805,0.097216,-0.271727,0.875060,-0.333300,0.116064,0.819487,0.223853,0.007116,0.746280,...,0.001716,0.731515,-0.187313,0.276628,-0.979238,1.156981,-0.110856,0.152424,15136.19,0


In [21]:
amt = list(set(dataset_train.Amount.values))
tags = list(set(dataset_train.Class.values))
len(tags) , len(amt)

(1, 268206)

In [26]:
# Count occurrences of each POS tag
tags_count = dict(dataset_train['Amount'].value_counts())

# Count occurrences of each word for each POS tag
tags_to_word_count = dataset_train.groupby(['Amount']).apply(lambda grp: grp.groupby('Amount')['tags'].count().to_dict()).to_dict()

# Count initial occurrences of each POS tag in each sentence
init_tags_count = dict(dataset_train.groupby('Class').first()['Amount'].value_counts())

# Initialize arrays for transition probabilities, emission probabilities, and other variables
tags_to_next_tags_count = np.zeros((len(tags), len(tags)), dtype=int)
sentences = list(dataset_train['Class'])
pos = list(dataset_train['Amount'])

# Calculate transition probabilities
for i in range(len(sentences)):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        tags_to_next_tags_count[prevtagid][nexttagid] += 1

my_start_prob = np.zeros((len(tags),))
my_transmat = np.zeros((len(tags), len(tags)))
my_emission_prob = np.zeros((len(tags), len(words)))
num_sentences = sum(init_tags_count.values())
sum_tags_to_next_tags = np.sum(tags_to_next_tags_count, axis=1)

# Calculate start probabilities, emission probabilities, and transition probabilities
for tag, tagid in tag2id.items():
    float_count_tag = float(tags_count.get(tag, 0))
    my_start_prob[tagid] = init_tags_count.get(tag, 0) / num_sentences
    for word, wordid in word2id.items():
        my_emission_prob[tagid][wordid] = tags_to_word_count.get(tag, {}).get(word, 0) / float_count_tag
    for tag2, tagid2 in tag2id.items():
        my_transmat[tagid][tagid2] = tags_to_next_tags_count[tagid][tagid2] / sum_tags_to_next_tags[tagid]


KeyError: 'Column not found: tags'

In [24]:
# Initialize the Hidden Markov Model
model = hmm.MultinomialHMM(n_components=len(tags), algorithm='viterbi', random_state=42)

# Set start probabilities, transition probabilities, and emission probabilities
model.startprob_ = my_start_prob
model.transmat_ = my_transmat
model.emissionprob_ = my_emission_prob


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


NameError: name 'my_start_prob' is not defined